# Stock Market Question Answering Agent

- answer questions about individual stocks using (mostly) OpenBB functions
- uses the OpenAPI tools functionality
- define a set of functions
- when prompting, give OpenAI the set of functions and their descriptions in a specified format
- if OpenAI needs to run a function to respond to the prompt, it will provide the function signature and ask you to provide the return values before continuing
  

In [1]:
import os
import sys
import dotenv
import warnings
from datetime import datetime, timedelta
import re

import pandas as pd

import IPython
from IPython.display import HTML, Image, Markdown, display

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
# use firefox because it updates less often, can disable updates
# recommend importing profile from Chrome for cookies, passwords
# looks less like a bot with more user cruft in the profile
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service

import base64
import requests
import json

import openbb
from openbb import obb
from openbb_core.app.model.obbject import OBBject

import openai
from openai import OpenAI
import tiktoken

import langchain
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

import wikipedia

# paid API for edgar filings
import sec_api
from sec_api import QueryApi, ExtractorApi

# free API for edgar filing
from sec_downloader import Downloader
import sec_parser as sp

dotenv.load_dotenv()

from BB_agent_tool import BB_agent_tool, get_response, agent_query

import pdb

# turn off excessive warnings
warnings.filterwarnings('ignore')


In [2]:
print(f'pandas         {pd.__version__}')
print(f'obb            {obb.system.version}')
print(f'selenium       {selenium.__version__}')
print(f'openai         {openai.__version__}')
print(f'langchain      {langchain.__version__}')
print(f'wikipedia      {wikipedia.__version__}')


pandas         2.2.2
obb            4.1.7
selenium       4.20.0
openai         1.28.0
langchain      0.1.20
wikipedia      (1, 4, 0)


# Connect to OpenBB

In [3]:
obb

OpenBB Platform v4.1.7

Utilities:
    /account
    /user
    /system
    /coverage

Routers:
    /commodity
    /crypto
    /currency
    /derivatives
    /econometrics
    /economy
    /equity
    /etf
    /fixedincome
    /index
    /news
    /quantitative
    /regulators
    /technical

Extensions:
    - commodity@1.0.4
    - crypto@1.1.5
    - currency@1.1.5
    - derivatives@1.1.5
    - econometrics@1.1.5
    - economy@1.1.5
    - equity@1.1.5
    - etf@1.1.5
    - fixedincome@1.1.5
    - index@1.1.5
    - news@1.1.5
    - quantitative@1.1.5
    - regulators@1.1.5
    - technical@1.1.6

    - alpha_vantage@1.1.5
    - benzinga@1.1.5
    - biztoc@1.1.5
    - cboe@1.1.5
    - ecb@1.1.5
    - federal_reserve@1.1.5
    - finra@1.1.5
    - finviz@1.0.4
    - fmp@1.1.5
    - fred@1.1.5
    - government_us@1.1.5
    - intrinio@1.1.5
    - nasdaq@1.1.6
    - oecd@1.1.5
    - polygon@1.1.5
    - sec@1.1.5
    - seeking_alpha@1.1.5
    - stockgrid@1.1.5
    - tiingo@1.1.5
    - tmx@1.0.2
 

In [4]:
# login with email and password
obb.account.login(email=os.environ['OPENBB_USER'], password=os.environ['OPENBB_PW'], remember_me=True)

In [5]:
obb.user

UserSettings

id: 06647c2f-ae77-71fb-8000-b4f607f65da0
profile: {'hub_session': {'username': 'drucev', 'email': 'drucev@hotmail.com', 'primary_usage': 'personal', 'user_uuid': 'c866b4d2-c09b-4b13-abb7-a93f1ac3c2b7', 'token_type': 'bearer', 'access_token': SecretStr('**********')}}
credentials: {'biztoc_api_key': SecretStr('**********'), 'fmp_api_key': SecretStr('**********'), 'intrinio_api_key': None, 'tradier_account_type': None, 'fred_api_key': SecretStr('**********'), 'polygon_api_key': SecretStr('**********'), 'tiingo_token': SecretStr('**********'), 'alpha_vantage_api_key': SecretStr('**********'), 'tradier_api_key': None, 'tradingeconomics_api_key': None, 'benzinga_api_key': None, 'nasdaq_api_key': None}
preferences: {'cache_directory': '/Users/drucev/OpenBBUserData/cache', 'chart_style': 'dark', 'data_directory': '/Users/drucev/OpenBBUserData', 'export_directory': '/Users/drucev/OpenBBUserData/exports', 'metadata': True, 'output_type': 'OBBject', 'plot_enable_pywry': True, 'plot

In [6]:
obb.account

/account
    login
    logout
    save
    refresh
    

In [7]:
# Change a credential - only need once, gets stored in openbb cloud
# obb.user.credentials.polygon_api_key = os.environ['POLYGON_API_KEY']
# obb.user.credentials.alpha_vantage_api_key = os.environ['ALPHAVANTAGE_API_KEY']
# obb.user.credentials.fred_api_key = os.environ['FRED_API_KEY']
# obb.user.credentials.tiingo_token = os.environ['TIINGO_API_KEY']
# obb.user.credentials.fmp_api_key = os.environ['FMP_API_KEY']
# obb.user.credentials.biztoc_api_key = os.environ['BIZTOC_API_KEY']

# Save account changes to the Hub
# obb.account.save()

# Refresh account with latest changes since login
# obb.account.refresh()

# Logout
# obb.account.logout()

In [8]:
obb.equity

/equity
    /calendar
    /compare
    /darkpool
    /discovery
    /estimates
    /fundamental
    market_snapshots
    /ownership
    /price
    profile
    screener
    search
    /shorts
    

In [9]:
obb.equity.search("Merck", provider="nasdaq").to_df().head(3)


,symbol,name,nasdaq_traded,exchange,etf,round_lot_size,test_issue,cqs_symbol,nasdaq_symbol,next_shares
0,MRK,"Merck & Company, Inc. Common Stock (new)",Y,N,N,100.0,N,MRK,MRK,N


In [10]:
symbol = "MRK"
company = "Merck"

In [11]:
obj = obb.equity.price.performance(symbol)
obj


OBBject

id: 06647c2f-b33b-7224-8000-39fced18adec
results: [{'symbol': 'MRK', 'one_day': 0.0024, 'wtd': None, 'one_week': 0.0087, 'mt...
provider: finviz
warnings: None
chart: None
extra: {'metadata': {'arguments': {'provider_choices': {'provider': 'finviz'}, 'sta...

In [12]:
obj.results


[FinvizPricePerformanceData(symbol=MRK, one_day=0.0024, wtd=None, one_week=0.0087, mtd=None, one_month=0.047599999999999996, qtd=None, three_month=0.026600000000000002, six_month=0.2944, ytd=0.2034, one_year=0.1432, two_year=None, three_year=None, four_year=None, five_year=None, ten_year=None, max=None, volatility_week=0.0125, volatility_month=0.0147, price=131.19, volume=7355209.0, average_volume=8220000.000000001, relative_volume=0.9, analyst_recommendation=None, analyst_score=1.5)]

In [13]:
obj.dict()


{'id': '06647c2f-b33b-7224-8000-39fced18adec',
 'results': [{'symbol': 'MRK',
   'one_day': 0.0024,
   'wtd': None,
   'one_week': 0.0087,
   'mtd': None,
   'one_month': 0.047599999999999996,
   'qtd': None,
   'three_month': 0.026600000000000002,
   'six_month': 0.2944,
   'ytd': 0.2034,
   'one_year': 0.1432,
   'two_year': None,
   'three_year': None,
   'four_year': None,
   'five_year': None,
   'ten_year': None,
   'max': None,
   'volatility_week': 0.0125,
   'volatility_month': 0.0147,
   'price': 131.19,
   'volume': 7355209.0,
   'average_volume': 8220000.000000001,
   'relative_volume': 0.9,
   'analyst_recommendation': None,
   'analyst_score': 1.5}],
 'provider': 'finviz',
 'warnings': None,
 'chart': None,
 'extra': {'metadata': {'arguments': {'provider_choices': {'provider': 'finviz'},
    'standard_params': {'symbol': 'MRK'},
    'extra_params': {}},
   'duration': 197146750,
   'route': '/equity/price/performance',
   'timestamp': datetime.datetime(2024, 5, 17, 16, 50

In [14]:
# use REST API on server running locally
# uvicorn openbb_core.api.rest_api:app --host 0.0.0.0 --port 8000 --reload
# REST API documentation - http://127.0.0.1:8000/docs
# openapi.json : http://127.0.0.1:8000/openapi.json

# not turning on authentication
# msg = "some_user:some_pass"
# msg_bytes = msg.encode('ascii')
# base64_bytes = base64.b64encode(msg_bytes)
# base64_msg = base64_bytes.decode('ascii')

url = f"http://127.0.0.1:8000/api/v1/equity/price/quote?provider=yfinance&symbol={symbol}"
# headers = {"accept": "application/json", "Authorization": f"Basic {base64_msg}"}
headers = {"accept": "application/json"}

response = requests.get(url=url, headers=headers)

response.json()


{'results': [{'symbol': 'MRK',
   'asset_type': 'EQUITY',
   'name': 'Merck & Co., Inc.',
   'exchange': 'NYQ',
   'bid': 131.27,
   'bid_size': 1000,
   'ask': 130.83,
   'ask_size': 800,
   'last_price': 131.19,
   'open': 130.69,
   'high': 131.32,
   'low': 129.81,
   'volume': 6956134,
   'prev_close': 130.88,
   'year_high': 133.1,
   'year_low': 99.14,
   'ma_50d': 126.9606,
   'ma_200d': 114.69035,
   'volume_average': 8217112.0,
   'volume_average_10d': 6707380.0,
   'currency': 'USD'}],
 'provider': 'yfinance',
 'warnings': None,
 'chart': None,
 'extra': {'metadata': {'arguments': {'provider_choices': {'provider': 'yfinance'},
    'standard_params': {'symbol': 'MRK'},
    'extra_params': {'use_cache': True, 'source': 'iex'}},
   'duration': 203261542,
   'route': '/equity/price/quote',
   'timestamp': '2024-05-17T16:50:03.230309'}}}

In [15]:
data = obb.equity.price.historical(symbol, provider="polygon")
data.to_dataframe()


,open,high,low,close,volume,vwap,transactions
date,,,,,,,
2023-05-17,116.37,116.655,113.4800,114.76,7210383.0,114.5498,78904
2023-05-18,114.14,115.070,113.3300,114.00,6437144.0,114.0864,75952
2023-05-19,114.33,116.240,114.1000,115.49,7647080.0,115.4905,63052
2023-05-22,115.48,117.080,114.3300,114.49,5607422.0,115.0297,75291
2023-05-23,113.97,114.790,113.0500,113.27,7733023.0,113.5640,88433
...,...,...,...,...,...,...,...
2024-05-10,130.82,130.880,129.9500,130.06,5540149.0,130.3223,73151
2024-05-13,129.51,129.710,128.0200,129.29,6920607.0,128.9122,85432
2024-05-14,129.20,129.200,128.0269,128.66,6055317.0,128.5396,76232


In [16]:
# use the local rest server
data = []
symbol2="SPY"
url = f"http://127.0.0.1:8000/api/v1/equity/price/historical?provider=polygon&symbol={symbol2}"
headers = {"accept": "application/json"}

response = requests.get(url, headers=headers, timeout=3)

if response.status_code == 200:
  data = OBBject.model_validate(response.json())

data.to_df()


,close,high,low,open,transactions,volume,vwap
date,,,,,,,
2023-05-17,415.23,415.8550,410.635,412.35,585038,86786957.0,413.4302
2023-05-18,419.23,419.6700,414.670,414.90,672492,97177195.0,417.1828
2023-05-19,418.62,420.7200,417.350,420.17,642871,103793317.0,419.0232
2023-05-22,418.79,420.3900,417.350,418.64,493531,60717899.0,419.0370
2023-05-23,414.09,418.7220,413.680,417.08,624936,86383493.0,415.8938
...,...,...,...,...,...,...,...
2024-05-10,520.84,522.6350,519.590,521.81,428536,52201942.0,520.8834
2024-05-13,520.91,522.6700,519.740,522.56,360847,36692999.0,521.0738
2024-05-14,523.30,523.8300,520.560,521.11,417233,57521901.0,522.2996


In [17]:
results = obb.equity.search(query='marvell', is_symbol=False, provider='nasdaq', use_cache=True)
[(r.symbol, r.name) for r in results.results]


[('MRVL', 'Marvell Technology, Inc. - Common Stock')]

In [18]:
# multiple symbols
quotes = obb.equity.price.quote("td,schw,jpm,ms", provider="fmp")
quotes.to_df()


,symbol,name,exchange,last_price,last_timestamp,open,high,low,volume,prev_close,...,year_high,year_low,price_avg50,price_avg200,avg_volume,market_cap,shares_outstanding,eps,pe,earnings_announcement
0,TD,The Toronto-Dominion Bank,NYSE,57.24,2024-05-17 20:00:01+00:00,57.02,57.380,56.7150,1669389,56.88,...,66.150,54.12,58.5264,60.07160,3019946,1.014350e+11,1772100000,4.64,12.34,2024-02-29 11:30:12+00:00
1,SCHW,The Charles Schwab Corporation,NYSE,78.78,2024-05-17 20:00:02+00:00,78.30,79.220,77.7700,6429451,78.04,...,79.220,48.66,72.4278,63.36110,7082288,1.400141e+11,1777280000,2.39,32.96,2024-07-16 12:30:00+00:00
2,JPM,JPMorgan Chase & Co.,NYSE,204.79,2024-05-17 20:00:08+00:00,203.81,205.045,202.8149,8991128,202.47,...,205.045,134.40,193.2614,166.88720,8845098,5.880893e+11,2871670000,16.56,12.37,2024-07-12 12:30:00+00:00
3,MS,Morgan Stanley,NYSE,100.22,2024-05-17 20:00:02+00:00,99.90,100.300,99.3040,3544176,99.58,...,100.980,69.42,92.1354,85.88715,7878209,1.628735e+11,1625160000,5.50,18.22,2024-07-16 12:30:00+00:00


In [19]:
# multiple providers

df = pd.DataFrame()

df["yfinance"] = (
  obb.equity.fundamental.balance(symbol, provider="yfinance", limit=3)
  .to_df().get("total_assets")
)

df["fmp"] = (
  obb.equity.fundamental.balance(symbol, provider="fmp", limit=3)
  .to_df().get("total_assets")
)

df["polygon"] = (
  obb.equity.fundamental.balance(symbol, provider="polygon", limit=3)
  .to_df().get("total_assets")
)

df

,yfinance,fmp,polygon
0,1.066750e+11,1.066750e+11,1.066750e+11
1,1.091600e+11,1.091600e+11,1.091600e+11
2,1.056940e+11,1.056940e+11,1.056940e+11
3,9.158800e+10,NaN,NaN


In [20]:
obb.news.company(symbol, provider='polygon', limit=5).to_df()


,title,text,images,url,symbols,source,id,amp_url,publisher
date,,,,,,,,,
2024-05-16 10:49:00+00:00,"The Zacks Analyst Blog Highlights Broadcom, Me...","Broadcom, Merck, Airbnb, ONEOK and PG&E are in...",[{'url': 'https://staticx-tuner.zacks.com/imag...,https://www.zacks.com/stock/news/2274496/the-z...,"PCG,MRK,OKE,AVGO,ABNB",Zacks Equity Research,rSveOM5z_sG_3ByZLLEbdQGcEV1fgmsxp81zzDegDYo,https://www.zacks.com/amp/stock/news/2274496/t...,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-16 13:35:03+00:00,Merck (MRK) Crossed Above the 20-Day Moving Av...,When a stock breaks out above the 20-day simpl...,[{'url': 'https://staticx-tuner.zacks.com/imag...,https://www.zacks.com/stock/news/2274678/merck...,MRK,Zacks Equity Research,PvosUfrWiZe-WIwDjbx2DeQZtC6PTFYafAk51SyoR4Y,https://www.zacks.com/amp/stock/news/2274678/m...,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-16 14:13:00+00:00,3 Drug Stocks to Watch on Raised 2024 Earnings...,"Here we discuss three drugmakers, Eli Lilly an...",[{'url': 'https://staticx-tuner.zacks.com/imag...,https://www.zacks.com/stock/news/2274825/3-dru...,"NVO,MRK,LLY",Kinjel Shah,xUp-v0S3tHM2CYzTNRJh_twmk1_qaHGKfGicQupe7HM,https://www.zacks.com/amp/stock/news/2274825/3...,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-17 15:10:00+00:00,5 Stocks Powering the Dow ETF Year to Date,"The Dow Jones touched the milestone of 40,000 ...",[{'url': 'https://staticx-tuner.zacks.com/imag...,https://www.zacks.com/stock/news/2275517/5-sto...,"AMZN,GS,MRK,WMT,AXP,DIA",Sweta Killa,zg2zh4fQJkCT2JKqgPpqUcBsQlVM-tWk3ao9UBYB5zA,https://www.zacks.com/amp/stock/news/2275517/5...,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-17 15:39:00+00:00,"Pharma Stock Roundup: BAYRY's Q1 Earnings, JNJ...",Bayer (BAYRY) releases Q1 results. J&J (JNJ) s...,[{'url': 'https://staticx-tuner.zacks.com/imag...,https://www.zacks.com/stock/news/2275562/pharm...,"JNJ,NVO,MRK,LLY,BAYRY",Kinjel Shah,stgIxTToXipTjGoYXkdRtTwSj9wP7mSF2euIUnFYIGk,https://www.zacks.com/amp/stock/news/2275562/p...,{'favicon_url': 'https://s3.polygon.io/public/...


# Prompt OpenAI 

In [21]:
MODEL = "gpt-4o"

# MAX_INPUT_TOKENS = 65536     
MAX_OUTPUT_TOKENS = 4096    # max in current model
MAX_RETRIES = 3
TEMPERATURE = 0

client = OpenAI()


In [22]:
messages = [{"role": "user", "content": "what is the airspeed velocity of an unladen swallow?"}]
response = get_response(client, messages, tools=[])
response_str = response.choices[0].message.content
response_str = response_str.replace("$", "\\\$")

print(response_str)

The question about the airspeed velocity of an unladen swallow is a famous line from the movie "Monty Python and the Holy Grail." In the context of the film, it's a humorous way to befuddle a bridge keeper demanding answers to arbitrary questions.

However, if you are looking for a more factual answer, it's important to specify which type of swallow you are asking about, as there are many species. Generally, the commonly referenced swallow in this context is the European Swallow (Hirundo rustica).

For the European Swallow, also known as the Barn Swallow, a commonly cited estimate for its cruising airspeed is roughly 11 meters per second, or approximately 24 miles per hour. 

It's worth noting that this is a rough estimate and actual speeds can vary based on a multitude of factors including wind conditions and whether the bird is flying to migrate, hunt, or navigate around obstacles.


# Question Answering Agent

In [23]:
# make a bespoke tool 

def get_10k_item1_from_symbol(symbol):
    """
    Get item 1 of the latest 10-K annual report filing for a given symbol.

    Args:
        symbol (str): The symbol of the equity.

    Returns:
        str: The item 1 of the latest 10-K annual report filing, or None if not found.

    """
    item1_text = None
    try:
        # sec needs you to identify yourself for rate limiting
        dl = Downloader(os.getenv("SEC_FIRM"), os.getenv("SEC_USER"))
        html = dl.get_filing_html(ticker=symbol, form="10-K")
        elements: list = sp.Edgar10QParser().parse(html)
        tree = sp.TreeBuilder().build(elements)
        sections = [n for n in tree.nodes if n.text.startswith("Item")]
        item1_node = sections[0]
        item1_text = "\n".join([n.text for n in sections[0].get_descendants()])
    except:
        return None
    # always return a list of dicts
    return [{'item1': item1_text}]

fn_metadata = {
    "name": "get_10k_item1_from_symbol",
    "description": "Given a stock symbol, gets item 1 of the company's latest 10-K annual report filing.",
    "openapi_path" : None,
    "callable": get_10k_item1_from_symbol,
    "parameters": {
        "symbol": {
            "type": "string",
            "description": "The symbol to get the 10-K item 1 for"
            }
        },
    "example_parameter_values": [{
        "symbol": "MSFT",
    }],
}

tool = BB_agent_tool(**fn_metadata)
tool_response = tool(symbol="MSFT")
len(json.loads(tool_response)[0]['item1'])


143226

In [24]:
user_message = "Please summarize item 1 from the latest MSFT annual report"
agent_query(client, user_message, verbose=False)


get_10k_item1_from_symbol({'symbol': 'MSFT'}) -> [{"item1": "Note About Forward-Looking Statements\nThis report includes estimates, projections, stat...


### Summary of Item 1 from Microsoft’s Latest Annual Report (10-K)

#### **Business Overview**
- **Company Mission**: Microsoft aims to empower every person and organization on the planet to achieve more through technology.
- **Core Strengths**:
  - **Artificial Intelligence (AI)**: Leading in AI-driven platforms and tools for various sectors, enhancing competitiveness, public services efficiency, and education.
  - **Cloud Solutions**: Comprehensive cloud offerings through Microsoft Azure and other cloud services, integrating AI and hybrid infrastructure.
  - **Productivity and Business Solutions**: Office 365, Dynamics 365, and LinkedIn services drive productivity and collaboration.
  - **Personal Computing**: Windows OS, Surface devices, Xbox gaming consoles, and Microsoft Edge browser enhance user interaction with technology.

#### **Business Segments**
1. **Productivity and Business Processes**:
   - **Office**: Office 365, Microsoft Teams, Dynamics 365.
   - **LinkedIn**: Talent Solutions, Marketing Solutions, Sales Solutions.

2. **Intelligent Cloud**:
   - **Azure**: Cloud computing platform.
   - **Server Products and Cloud Services**: SQL Server, Windows Server, Visual Studio.
   - **Enterprise Services**: Consulting services.

3. **More Personal Computing**:
   - **Windows**: Operating system lines (OEM licensing, commercial licensing).
   - **Devices**: Surface tablets, other PC accessories.
   - **Gaming**: Xbox hardware, Xbox Game Pass.
   - **Search and News Advertising**: Bing, Microsoft News.

#### **Strategic and Research Focus**
- **AI & Automation**: Significant focus on AI integration across products and services including Microsoft Teams, Dynamics 365, and Azure AI.
- **Security & Compliance**: Emphasis on security measures, compliance, and end-to-end solutions to support customer needs and regulatory requirements.
- **Environmental Sustainability**: Commitment to sustainability with goals to be carbon negative, water positive, and zero waste by 2030.

#### **Market Position and Competition**
- Competes against global tech giants like Apple, Google, Amazon, and various open-source software products in multiple sectors including cloud services, AI, personal computing, and gaming.
- Differentiates through integrated and comprehensive solutions across cloud, AI, and productivity tools, designed to meet varied industry needs.

#### **Key Risks**
- **Intense Competition**: Innovations by competitors, regulatory pressures, and evolving customer needs pose continuous challenges.
- **Cybersecurity Threats**: Increasingly sophisticated cyber threats and security vulnerabilities.
- **Regulatory Hurdles**: Compliance with global data privacy and security regulations, antitrust scrutiny, and trade laws.
- **Operational Risks**: Dependence on critical infrastructure and supply chain continuity for hardware products and cloud services.

#### **Corporate Responsibility**
- Focused on social, economic, and environmental sustainability.
- Project initiatives aimed at racial equity, economic opportunity, and digital skills enhancement globally.

#### **Management and Corporate Governance**
- A leadership team dedicated to driving long-term strategic goals and maintaining robust corporate governance principles. 

Overall, Microsoft is positioning itself as a leader in AI and cloud solutions while continually addressing the needs of its global customer base through innovative, secure, and sustainable technologies.

# Add more OpenBB tools
Map OpenBB functions to OpenAI tools

In [25]:
# TODO:

# move object to a file , move llm calls to a file

# make a streamlit app that maintains conversation

# use assistants API


In [26]:
# load the OpenAPI / swagger spec from
# http://127.0.0.1:8000/openapi.json
with open("openapi.json", 'r') as file:
    data = json.load(file)

str(data)[:2000]

"{'openapi': '3.1.0', 'info': {'title': 'OpenBB Platform API', 'description': 'This is the OpenBB Platform API.', 'termsOfService': 'http://example.com/terms/', 'contact': {'name': 'OpenBB Team', 'url': 'https://openbb.co/', 'email': 'hello@openbb.co'}, 'license': {'name': 'MIT', 'url': 'https://github.com/OpenBB-finance/OpenBBTerminal/blob/develop/LICENSE'}, 'version': '1'}, 'servers': [{'url': 'http://localhost:8000', 'description': 'Local OpenBB development server'}], 'paths': {'/api/v1/commodity/lbma_fixing': {'get': {'tags': ['commodity'], 'summary': 'Lbma Fixing', 'description': 'Daily LBMA Fixing Prices in USD/EUR/GBP.', 'operationId': 'commodity_lbma_fixing', 'parameters': [{'name': 'provider', 'in': 'query', 'required': False, 'schema': {'enum': ['nasdaq'], 'const': 'nasdaq', 'type': 'string', 'default': 'nasdaq', 'title': 'Provider'}}, {'name': 'asset', 'in': 'query', 'required': False, 'schema': {'enum': ['gold', 'silver'], 'type': 'string', 'description': 'The metal to get 

In [27]:
# list all the equity functions
for path_str, fn_json in data['paths'].items():
    if path_str.find('equity') != -1:
        print(path_str)


/api/v1/equity/calendar/ipo
/api/v1/equity/calendar/dividend
/api/v1/equity/calendar/splits
/api/v1/equity/calendar/earnings
/api/v1/equity/compare/peers
/api/v1/equity/compare/groups
/api/v1/equity/estimates/price_target
/api/v1/equity/estimates/historical
/api/v1/equity/estimates/consensus
/api/v1/equity/estimates/analyst_search
/api/v1/equity/estimates/forward_sales
/api/v1/equity/estimates/forward_eps
/api/v1/equity/darkpool/otc
/api/v1/equity/discovery/gainers
/api/v1/equity/discovery/losers
/api/v1/equity/discovery/active
/api/v1/equity/discovery/undervalued_large_caps
/api/v1/equity/discovery/undervalued_growth
/api/v1/equity/discovery/aggressive_small_caps
/api/v1/equity/discovery/growth_tech
/api/v1/equity/discovery/top_retail
/api/v1/equity/discovery/upcoming_release_days
/api/v1/equity/discovery/filings
/api/v1/equity/fundamental/multiples
/api/v1/equity/fundamental/balance
/api/v1/equity/fundamental/balance_growth
/api/v1/equity/fundamental/cash
/api/v1/equity/fundamental/r

In [28]:
data['paths']['/api/v1/equity/search']


{'get': {'tags': ['equity'],
  'summary': 'Search',
  'description': 'Search for stock symbol, CIK, LEI, or company name.',
  'operationId': 'equity_search',
  'parameters': [{'name': 'provider',
    'in': 'query',
    'required': True,
    'schema': {'enum': ['cboe', 'intrinio', 'nasdaq', 'sec', 'tmx', 'tradier'],
     'type': 'string',
     'title': 'Provider'}},
   {'name': 'query',
    'in': 'query',
    'required': False,
    'schema': {'type': 'string',
     'description': 'Search query.',
     'default': '',
     'title': 'Query'},
    'description': 'Search query.'},
   {'name': 'is_symbol',
    'in': 'query',
    'required': False,
    'schema': {'type': 'boolean',
     'description': 'Whether to search by ticker symbol.',
     'default': False,
     'title': 'Is Symbol'},
    'description': 'Whether to search by ticker symbol.'},
   {'name': 'use_cache',
    'in': 'query',
    'required': False,
    'schema': {'anyOf': [{'type': 'boolean'}, {'type': 'null'}],
     'descriptio

In [29]:
fn_metadata = {
    "name": "get_equity_search_symbol",
    "description": "Given a search string, get the stock symbol of the top company whose name best matches the search string.",
    "openapi_path" : '/api/v1/equity/search',
    "parameters": {
        "query": {
            "type": "string",
            "description": "The search string to match to the stock symbol."
            },""
        "limit": {
            "type": "integer",
            "description": "The number of results to return. Pick a small number from 1 to 10 and choose the best response."
            }
        },
    "example_parameter_values": [{
        "query": "Broadcom",
    }],
    # "singular": 1,
}

tool = BB_agent_tool(**fn_metadata)



In [30]:
BB_agent_tool.agent_registry['get_equity_search_symbol'](query="Southwest Air")

'[{"symbol": "LUV", "name": "SOUTHWEST AIRLS CO COM", "dpm_name": "Susquehanna Securities, LLC", "post_station": "5/1"}]'

In [31]:
agent_query(client, "What is the stock symbol for Southwest airlines?", verbose=False)


get_equity_search_symbol({'query': 'Southwest Airlines', 'limit': 1}) -> []
get_equity_search_symbol({'query': 'Southwest', 'limit': 10}) -> [{"symbol": "CSWC", "name": "CAPITAL SOUTHWEST CORP COM", "dpm_name": "Wolverine Trading, LLC", "pos...


The stock symbol for Southwest Airlines is **LUV**.

In [32]:
agent_query(client, "What is the stock symbol for yakimee air?", verbose=False)


get_equity_search_symbol({'query': 'yakimee air', 'limit': 1}) -> []


I couldn't find a stock symbol for "yakimee air." It's possible there might be a spelling mistake or the company might not be publicly listed. Could you please provide more details or verify the spelling?

In [33]:
# agent is able to first get symbol, then get annual report for symbol 
agent_query(client, "What is item 1 from the 10k annual report for Microsfot?", verbose=True)


Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_dtFEGXIYmcZRf4WJcqNeFM2z', function=Function(arguments='{"query":"Microsoft","limit":1}', name='get_equity_search_symbol'), type='function')]))
[ChatCompletionMessageToolCall(id='call_dtFEGXIYmcZRf4WJcqNeFM2z', function=Function(arguments='{"query":"Microsoft","limit":1}', name='get_equity_search_symbol'), type='function')]
get_equity_search_symbol({'query': 'Microsoft', 'limit': 1}) -> [{"symbol": "MSFT", "name": "MICROSOFT CORP COM", "dpm_name": "Susquehanna Securities, LLC", "post_s...
Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_OASuFjCDr75htk1WbF9WAKAr', function=Function(arguments='{"symbol":"MSFT"}', name='get_10k_item1_from_symbol'), type='funct

For Microsoft (symbol: MSFT), Item 1 from the latest 10-K annual report includes a comprehensive overview of the company's business and operations, divided into several sections. Here is a summary of key points:

### General Overview
Microsoft's mission is to "empower every person and every organization on the planet to achieve more." The company aims to create local opportunity, growth, and impact globally by providing platforms and tools powered by artificial intelligence (AI). Their offerings span from infrastructure and data to business applications and collaboration tools.

### Products and Services
- **Cloud-based Solutions**: These include software, services, platforms, and content.
- **Online Advertising**: Microsoft delivers relevant online ads globally.
- **Devices**: They design and sell PCs, tablets, gaming and entertainment consoles, and intelligent devices.
- **Productivity and Collaboration Tools**: Office 365, Dynamics 365, LinkedIn, etc.
- **Microsoft Cloud**: It includes services like Azure, AI solutions, and hybrid cloud offerings.

### Strategic Ambitions
1. **Reinvent Productivity and Business Processes**: Includes products like Microsoft 365, Microsoft Teams, and Dynamics 365.
2. **Build the Intelligent Cloud and Intelligent Edge Platform**: Azure and AI capabilities are key focus areas.
3. **Create More Personal Computing**: Innovations in Windows, Microsoft Edge, Bing, and Xbox.

### Future Opportunities
Microsoft is focused on leveraging the breadth and depth of its cloud services while spearheading AI innovations responsibly under guiding principles such as fairness, reliability, safety, privacy, security, inclusiveness, transparency, and accountability.

### Corporate Social Responsibility
- **Sustainability**: Commitment to be a carbon-negative, water-positive, and zero-waste company by 2030.
- **Racial Equity Initiatives**: A multi-year plan focused on strengthening communities, engaging ecosystems, and increasing representation and inclusion.
- **Digital Skills Development**: Initiatives like "Skills for Jobs" aim to close the digital skills gap for job seekers globally.

### Human Capital Resources
Microsoft emphasizes recruiting and retaining a diverse and talented workforce, offering various learning opportunities, supporting employee wellbeing, and fostering an inclusive culture. They also maintain robust programs for compensation, pay equity, and career advancement.

This summary captures the essence of Item 1 from Microsoft’s latest 10-K report, highlighting the company's business strategy, operations, and social responsibility commitments.

In [ ]:
"What is the last market quote for Nvidia

In [34]:
agent_query(client, f"What is the last market quote for symbol {symbol}?", verbose=False)


I'm not equipped to retrieve real-time market quotes for stock symbols. For up-to-date market prices, you might want to consider checking financial news websites, stock market platforms, or using dedicated financial services like Bloomberg or Reuters.

In [35]:
agent_query(client, f"What is the last market quote for Consolidated Agglomerators", verbose=False)


get_equity_search_symbol({'query': 'Consolidated Agglomerators', 'limit': 10}) -> []


I couldn't find a ticker symbol for "Consolidated Agglomerators." It's possible that there might be a spelling error, the company might not be publicly listed, or it might be known by a different name.

Could you please provide more details or check the spelling? Alternatively, if you have another company or stock ticker in mind, please let me know!

In [36]:
agent_query(client, f"What is the current weather in Las Vegas", verbose=False)


I am unable to provide weather information. For current weather details, you can check a weather website or use a weather app.

In [37]:
tool = BB_agent_tool(
    name="get_company_profile_json",
    description="Given a stock symbol, get general background data about the company such as company name, industry, and sector data in JSON format",
    openapi_path='/api/v1/equity/profile',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

agent_query(client, f"Can you provide a basic company profile of symbol {symbol}", verbose=False)


get_company_profile_json({'symbol': 'MRK'}) -> [{"symbol": "MRK", "name": "Merck & Co Inc", "cik": null, "cusip": null, "isin": null, "lei": null, ...


### Company Profile: Merck & Co Inc (Symbol: MRK)

**Basic Information:**
- **Name:** Merck & Co Inc
- **Symbol:** MRK
- **Sector:** Healthcare
- **Industry Category:** Drug Manufacturers - General
- **Stock Exchange:** NYSE
- **Market Cap:** \\$332.28 Billion
- **Employees:** 72,000
- **Indexes:** DJIA, S&P 500

**Description:**
Merck & Co., Inc. is a healthcare company that provides health solutions through its prescription medicines, vaccines, biologic therapies, animal health, and consumer care products. 

**Segments:**
1. **Pharmaceutical:**
   - Human health pharmaceutical and vaccine products.
   
2. **Animal Health:**
   - Discovers, develops, manufactures, and markets animal health products, such as pharmaceutical and vaccine products, for the prevention, treatment, and control of disease in livestock and companion animal species.
   
3. **Other:**
   - Comprises sales for the non-reportable segments of healthcare services.

**Additional Information:**
- **Founded:** 1891
- **Headquarters:** Rahway, NJ
- **Institutional Ownership:** 79.08%
- **Beta:** 0.41
- **Shares Outstanding:** 2.53 Billion
- **Shares Float:** 2.53 Billion
- **Short Interest:** 18.75 Million

If you need more detailed information, just let me know!

In [38]:
tool = BB_agent_tool(
    name="get_equity_shorts_short_interest",
    description="Given a stock symbol, get data on short volume and days to cover in JSON format.",
    openapi_path='/api/v1/equity/shorts/short_interest',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

agent_query(client, f"Provide latest statistics on short interest for symbol {symbol}", verbose=False)


get_equity_shorts_short_interest({'symbol': 'MRK'}) -> [{"settlement_date": "2021-07-15", "symbol": "MRK", "issue_name": "Merck & Co., Inc.", "market_class...


Here are the latest statistics on short interest for Merck & Co., Inc. (symbol: MRK):

- **Settlement Date:** November 30, 2023
  - **Short Position:** 16,691,624 shares
  - **Previous Short Position:** 18,021,513 shares
  - **Average Daily Volume:** 9,408,394 shares
  - **Days to Cover:** 1.77 days
  - **Change:** -1,329,889 shares
  - **Change Percentage:** -7.38%

- **Settlement Date:** December 15, 2023
  - **Short Position:** 13,467,306 shares
  - **Previous Short Position:** 16,691,624 shares
  - **Average Daily Volume:** 10,762,231 shares
  - **Days to Cover:** 1.25 days
  - **Change:** -3,224,318 shares
  - **Change Percentage:** -19.32%

- **Settlement Date:** December 29, 2023
  - **Short Position:** 17,866,683 shares
  - **Previous Short Position:** 13,467,306 shares
  - **Average Daily Volume:** 6,974,790 shares
  - **Days to Cover:** 2.56 days
  - **Change:** 4,399,377 shares
  - **Change Percentage:** 32.67%

- **Settlement Date:** January 31, 2024
  - **Short Position:** 17,017,794 shares
  - **Previous Short Position:** 20,814,150 shares
  - **Average Daily Volume:** 7,839,892 shares
  - **Days to Cover:** 2.17 days
  - **Change:** -3,796,356 shares
  - **Change Percentage:** -18.24%

- **Settlement Date:** February 15, 2024
  - **Short Position:** 16,818,374 shares
  - **Previous Short Position:** 17,017,794 shares
  - **Average Daily Volume:** 7,441,783 shares
  - **Days to Cover:** 2.26 days
  - **Change:** -199,420 shares
  - **Change Percentage:** -1.17%

These statistics show the short interest in MRK over the last few settlement periods, indicating fluctuations in the number of shorted shares and average daily volume.

In [39]:
tool = BB_agent_tool(
    name="get_equity_fundamental_historical_splits",
    description="Given a stock symbol, get the company's historical stock splits in JSON format.",
    openapi_path='/api/v1/equity/fundamental/historical_splits',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)


In [40]:
agent_query(client, f"Provide historical split information for symbol AAPL", verbose=False)

get_equity_fundamental_historical_splits({'symbol': 'AAPL'}) -> [{"date": "2020-08-31", "numerator": 4.0, "denominator": 1.0, "split_ratio": null, "label": "August ...


Here is the historical stock split information for Apple Inc. (symbol: AAPL):

1. August 31, 2020: 4-for-1 split
2. June 9, 2014: 7-for-1 split
3. February 28, 2005: 2-for-1 split
4. June 21, 2000: 2-for-1 split
5. June 16, 1987: 2-for-1 split

In [41]:
data["paths"]["/api/v1/equity/fundamental/balance"]

{'get': {'tags': ['equity'],
  'summary': 'Balance',
  'description': 'Get the balance sheet for a given company.',
  'operationId': 'equity_fundamental_balance',
  'parameters': [{'name': 'provider',
    'in': 'query',
    'required': True,
    'schema': {'enum': ['fmp', 'intrinio', 'polygon', 'yfinance'],
     'type': 'string',
     'title': 'Provider'}},
   {'name': 'symbol',
    'in': 'query',
    'required': True,
    'schema': {'type': 'string',
     'description': 'Symbol to get data for.',
     'title': 'Symbol'},
    'description': 'Symbol to get data for.'},
   {'name': 'period',
    'in': 'query',
    'required': False,
    'schema': {'type': 'string',
     'description': 'Time period of the data to return.',
     'default': 'annual',
     'title': 'Period'},
    'description': 'Time period of the data to return.'},
   {'name': 'limit',
    'in': 'query',
    'required': False,
    'schema': {'anyOf': [{'type': 'integer', 'minimum': 0}, {'type': 'null'}],
     'description':

In [42]:
tool = BB_agent_tool(
    name="get_balance_sheet_json",
    description="Given a stock symbol, get the latest balance sheet data with assets and liabilities for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/balance',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
    },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
    singular=1
)

agent_query(client, f"what are the latest total assets for symbol {symbol}", verbose=False)


get_balance_sheet_json({'symbol': 'MRK'}) -> {"period_ending": "2023-12-31", "fiscal_period": "FY", "fiscal_year": 2023, "filing_date": "2024-04-...


As of the latest fiscal year ending on December 31, 2023, Merck & Co., Inc. (symbol: MRK) reported total assets of \\$106.675 billion.

For more detailed information, you can refer to their [latest 10-K filing](https://www.sec.gov/Archives/edgar/data/310158/000119312524093194/d807955d10ka.htm).

In [43]:
# can give by year
# obb.equity.fundamental.balance(symbol='NVDA', period='annual', fiscal_year='2023', limit=1)
# also growth from prior period obb.equity.fundamental.balance_growth

In [44]:
tool = BB_agent_tool(
    name="get_cash_flow_json",
    description="Given a stock symbol, get the latest cash flow statement data for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/cash',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
    singular=1
)

agent_query(client, f"what was the most recent cash flow from operations for stock symbol {symbol}", verbose=False)


get_cash_flow_json({'symbol': 'MRK'}) -> {"period_ending": "2023-12-31", "fiscal_period": "FY", "fiscal_year": 2023, "filing_date": "2024-04-...


The most recent cash flow from operations for Merck & Co., Inc. (symbol: MRK) is \\$13,006,000,000 USD, as of the fiscal year ending December 31, 2023.

For more detailed information, you can view their filing [here](https://www.sec.gov/Archives/edgar/data/310158/000119312524093194/d807955d10ka.htm).

In [45]:
tool.agent_registry

{'get_10k_item1_from_symbol': <BB_agent_tool.BB_agent_tool at 0x323434cd0>,
 'get_equity_search_symbol': <BB_agent_tool.BB_agent_tool at 0x1056dbbd0>,
 'get_company_profile_json': <BB_agent_tool.BB_agent_tool at 0x3231c7890>,
 'get_equity_shorts_short_interest': <BB_agent_tool.BB_agent_tool at 0x32a1951d0>,
 'get_equity_fundamental_historical_splits': <BB_agent_tool.BB_agent_tool at 0x327252810>,
 'get_balance_sheet_json': <BB_agent_tool.BB_agent_tool at 0x320016510>,
 'get_cash_flow_json': <BB_agent_tool.BB_agent_tool at 0x32241d850>}

In [46]:
tool = BB_agent_tool(
    name="get_income_statement_json",
    description="Given a stock symbol, get the latest income statement data for the company in JSON format",
    openapi_path='/api/v1/equity/fundamental/income',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
    singular=1
)

agent_query(client, f"what was the most recent net income for symbol {symbol}", verbose=False)


get_income_statement_json({'symbol': 'MRK'}) -> {"period_ending": "2023-12-31", "fiscal_period": "FY", "fiscal_year": 2023, "filing_date": "2024-04-...


The most recent net income for Merck & Co., Inc. (symbol: MRK) for the fiscal year ending December 31, 2023, was \\$365 million.

For more detailed information, you can view the complete filing [here](https://www.sec.gov/Archives/edgar/data/310158/000119312524093194/d807955d10ka.htm).

In [47]:
tool = BB_agent_tool(
    name="get_fundamental_metrics_json",
    description="Given a stock symbol, get fundamental metrics for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/metrics',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

agent_query(client, f"what was the most recent PE ratio for stock symbol {symbol}", verbose=False)


get_fundamental_metrics_json({'symbol': 'MRK'}) -> [{"symbol": "MRK", "market_cap": 332280000000.0, "pe_ratio": 146.04, "foward_pe": 13.2, "eps": 0.9, ...


The most recent PE ratio for Merck & Co., Inc. (symbol: MRK) is 146.04.

In [48]:
tool = BB_agent_tool(
    name="get_fundamental_ratios_json",
    description="Given a stock symbol, get fundamental valuation ratios for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/ratios',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
    singular=1
)

agent_query(client, f"what was the most recent price to sales ratio for stock symbol {symbol}", verbose=False)


get_fundamental_metrics_json({'symbol': 'MRK'}) -> [{"symbol": "MRK", "market_cap": 332280000000.0, "pe_ratio": 146.04, "foward_pe": 13.2, "eps": 0.9, ...


The most recent Price to Sales (P/S) ratio for Merck & Co., Inc. (symbol: MRK) is 5.44.

In [49]:
tool = BB_agent_tool(
    name="get_equity_fundamental_multiples",
    description="Given a stock symbol, get fundamental valuation multiples for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/multiples',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
    singular=1
)

agent_query(client, f"what was the most recent revenue per share for stock symbol {symbol}", verbose=False)


get_equity_search_symbol({'query': 'MRK', 'limit': 1}) -> [{"symbol": "FDT", "name": "FIRST TR EXCH TRD ALPHDX FD II DEV MRK EX US", "dpm_name": "Belvedere Tr...
get_equity_search_symbol({'query': 'Merck', 'limit': 1}) -> [{"symbol": "MRK", "name": "MERCK & CO INC COM", "dpm_name": "Wolverine Trading, LLC", "post_station...
get_equity_fundamental_multiples({'symbol': 'MRK'}) -> {"symbol": "MRK", "revenue_per_share_ttm": 24.25424397947098, "net_income_per_share_ttm": 0.91038294...


The most recent revenue per share for Merck & Co., Inc. (MRK) is \\$24.25 (trailing twelve months).

In [50]:
# tool = BB_agent_tool(
#     name="get_historical_eps",
#     description="Given a stock symbol, get historical earnings per share data for the company in JSON format.",
#     openapi_path='/api/v1/equity/fundamental/historical_eps',
#     parameters={
#         "symbol": {
#             "type": "string",
#             "description": "The stock symbol."
#         }
#         },
#     example_parameter_values=[{
#         "symbol": "NVDA",
#     }],
# )

# 

In [51]:
# agent_query(client, f"what was the EPS for {company} in the quarter ended 2022-09-30?", verbose=False)


In [52]:
tool = BB_agent_tool(
    name="get_equity_fundamental_dividend",
    description="Given a stock symbol, get the latest dividend data for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/dividends',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

agent_query(client, f"what was the latest dividend data for stock symbol {symbol}?", verbose=False)


get_equity_fundamental_dividend({'symbol': 'MRK'}) -> [{"ex_dividend_date": "2024-03-14", "amount": 0.77, "label": "March 14, 24", "adj_dividend": 0.77, "...


The latest dividend data for Merck & Co., Inc. (stock symbol: MRK) is as follows:

- **Ex-dividend Date**: March 14, 2024
- **Amount**: \\$0.77 per share
- **Record Date**: March 15, 2024
- **Payment Date**: April 5, 2024
- **Declaration Date**: January 23, 2024

If you need more detailed information or historical dividend data, feel free to ask!

In [53]:
agent_query(client, f"what was the dividend data as of the end of 2022 for stock symbol {symbol}?", verbose=False)


get_equity_search_symbol({'query': 'MRK', 'limit': 5}) -> [{"symbol": "FDT", "name": "FIRST TR EXCH TRD ALPHDX FD II DEV MRK EX US", "dpm_name": "Belvedere Tr...


It seems that "MRK" did not directly match any ticker symbols. However, the closest matches found are different than expected:

1. **FDT**: FIRST TR EXCH TRD ALPHDX FD II DEV MRK EX US
2. **BND**: VANGUARD BD INDEX FDS TOTAL BND MRKT

Could you please confirm that you're asking about Merck & Co., Inc., which is commonly referred to by the ticker symbol "MRK"? This will help ensure accurate information is provided.

In [54]:
tool = BB_agent_tool(
    name="get_trailing_dividend_yield_json",
    description="Given a stock symbol, get the 1 year trailing dividend yield for the company over time in JSON format.",
    openapi_path='/api/v1/equity/fundamental/trailing_dividend_yield',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

agent_query(client, f"what was the dividend yield for stock symbol {symbol} as of the end of 2023?", verbose=False)


get_equity_search_symbol({'query': 'MRK', 'limit': 1}) -> [{"symbol": "FDT", "name": "FIRST TR EXCH TRD ALPHDX FD II DEV MRK EX US", "dpm_name": "Belvedere Tr...
get_equity_search_symbol({'query': 'merck', 'limit': 1}) -> [{"symbol": "MRK", "name": "MERCK & CO INC COM", "dpm_name": "Wolverine Trading, LLC", "post_station...
get_trailing_dividend_yield_json({'symbol': 'MRK'}) -> [{"date": "2023-05-17", "trailing_dividend_yield": 0.0247472987}, {"date": "2023-05-18", "trailing_d...


As of the end of 2023, the trailing dividend yield for Merck & Co., Inc. (symbol: MRK) was approximately 2.72%.

In [55]:
tool = BB_agent_tool(
    name="get_price_performance_json",
    description="Given a stock symbol, get price performance data for the stock for different time periods in JSON format.",
    openapi_path='/api/v1/equity/price/performance',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

agent_query(client, f"what was the performance for {company} from 1 year ago?", verbose=False)


get_equity_search_symbol({'query': 'Merck', 'limit': 10}) -> [{"symbol": "MRK", "name": "MERCK & CO INC COM", "dpm_name": "Wolverine Trading, LLC", "post_station...
get_price_performance_json({'symbol': 'MRK'}) -> [{"symbol": "MRK", "one_day": 0.0024, "wtd": null, "one_week": 0.0087, "mtd": null, "one_month": 0.0...


The performance of Merck & Co Inc (symbol: MRK) over the past year has been as follows:

- **1-Year Performance:** +14.32%
- **Current Price:** \\$131.19
- **Volume:** 7,355,209
- **Average Volume:** 8,220,000
- **Volatility (Week):** 1.25%
- **Volatility (Month):** 1.47%

If you need more detailed information or specific time periods, please let me know!

In [56]:
# this might exceed token context making it unreliable
tool = BB_agent_tool(
    name="get_etf_equity_exposure_json",
    description="Given a stock symbol, get the exposure of ETFs to the stock in JSON format.",
    openapi_path='/api/v1/etf/equity_exposure',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)


In [57]:
pd.DataFrame(json.loads(tool(symbol='MRK'))).sort_values('weight', ascending=False)

,equity_symbol,etf_symbol,shares,weight,market_value
447,MRK,FTXH,11684.0,0.075700,1.519621e+06
13,MRK,XLV,18454434.0,0.062123,2.400184e+09
77,MRK,QDVG.DE,1126933.0,0.061660,1.457012e+08
78,MRK,IUHE.AS,0.0,0.061095,1.451633e+08
79,MRK,IUHC.L,0.0,0.061095,1.451633e+08
...,...,...,...,...,...
572,MRK,GXUS,2163.0,NaN,3.629300e+05
687,MRK,GPAL,640.0,NaN,6.631040e+04
698,MRK,GDEF,397.0,NaN,5.170131e+04
706,MRK,EUNU.DE,50000.0,NaN,3.868500e+04


In [58]:
agent_query(client, f"which ETF has the highest weight in {company} ?", verbose=False)


get_equity_search_symbol({'query': 'Merck', 'limit': 5}) -> [{"symbol": "MRK", "name": "MERCK & CO INC COM", "dpm_name": "Wolverine Trading, LLC", "post_station...
get_etf_equity_exposure_json({'symbol': 'MRK'}) -> [{"equity_symbol": "MRK", "etf_symbol": "VTSAX", "shares": 79663326.0, "weight": 0.0066, "market_val...


The ETF with the highest weight in Merck (MRK) is the **Health Care Select Sector SPDR Fund (XLV)**, with a weight of approximately **6.21%**.

In [59]:
[(k, t.description) for k, t in BB_agent_tool.agent_registry.items()]

[('get_10k_item1_from_symbol',
  "Given a stock symbol, gets item 1 of the company's latest 10-K annual report filing."),
 ('get_equity_search_symbol',
  'Given a search string, get the stock symbol of the top company whose name best matches the search string.'),
 ('get_company_profile_json',
  'Given a stock symbol, get general background data about the company such as company name, industry, and sector data in JSON format'),
 ('get_equity_shorts_short_interest',
  'Given a stock symbol, get data on short volume and days to cover in JSON format.'),
 ('get_equity_fundamental_historical_splits',
  "Given a stock symbol, get the company's historical stock splits in JSON format."),
 ('get_balance_sheet_json',
  'Given a stock symbol, get the latest balance sheet data with assets and liabilities for the company in JSON format.'),
 ('get_cash_flow_json',
  'Given a stock symbol, get the latest cash flow statement data for the company in JSON format.'),
 ('get_income_statement_json',
  'Give

In [60]:
# for k, t in BB_agent_tool.agent_registry.items():
#     print(t.example_code)

In [61]:
len(" ".join([(t.example_code) for k, t in BB_agent_tool.agent_registry.items()]))

23700